# HNU CE 프로그래밍언어론 (2023년 1학기) HW2
중간고사 프로그래밍 과제 파트입니다.

- 이름: 최영수
- 학번: 20190633

이름과 학번을 위에 작성하시오.


적절한 내용과 개수의 테스트를 작성하지 않으면 감점. 테스트가 적절하지 않거나 지나치게 부족하면 함수를 올바르게 작성했더라도 점수를 얻지 못할 수 있음에 유의!

----

In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables FlexibleInstances TypeApplications RankNTypes KindSignatures #-}

import Data.List ( nub, elemIndex, (\\) ) -- Data.List 에서 혹시 더 필요해서 활용할 함수가 있다면 여기에 추가로 import 해오면 됨

In [2]:
data Expr = I Int          -- n
          | Add Expr Expr  -- e + e
       deriving (Eq,Ord,Show) -- Eq 같은지, Ord 순서 비교, Show 출력 가능

## 문제1. (7장 연습문제 4번의 $\stackrel{_R}{\longmapsto}$에 해당하는 의미구조 계산기 함수) 15점
함수를 작성하고 주어진 식에 대한 계산이 더 이상 진행되지 않을 때까지 계산하는 적절한 테스트를 두 개 이상의 산술식(`Expr`)에 대해 작성해야 함

In [3]:
-- stepR :: Expr -> [Expr] -- 여기에 적절히 작성
-- stepR' :: [Expr] -> [Expr] -- 여기에 적절히 작성
import Data.List (nub)
stepR :: Expr -> [Expr]
stepR (I n1) = []
stepR (I n1 `Add` I n2) = [I n] where n = n1 + n2
stepR (e1 `Add` I n2) = [Add e1' (I n2) | e1' <- stepR e1]
stepR (e1 `Add` e2) = [Add e1' e2 | e1' <- stepR e1] ++ [Add e1 e2' | e2' <- stepR e2]

stepR' :: [Expr] -> [Expr]
stepR' es = nub (concat [stepR e | e <- es])

In [4]:
-- 적절히 테스트 2개 이상
e0 = (I 3)
e1 = (Add (I 1) (I 2))
e2 = (Add (I 2) (Add (I 2) (I 3)))
e3 = (Add (Add (I 3) (I 4)) (Add (I 5) (I 6)))
[e0]

[e1]
stepR' it

[e2]
stepR' it
stepR' it

[e3]
stepR' it
stepR' it
stepR' it

[I 3]

[Add (I 1) (I 2)]

[I 3]

[Add (I 2) (Add (I 2) (I 3))]

[Add (I 2) (I 5)]

[I 7]

[Add (Add (I 3) (I 4)) (Add (I 5) (I 6))]

[Add (I 7) (Add (I 5) (I 6)),Add (Add (I 3) (I 4)) (I 11)]

[Add (I 7) (I 11)]

[I 18]

## 문제2. (7장 연습문제 5번) 20점
우선 7장 연습문제 4번의 나머지 부분을 해결해서 생각을 정리한 다음 작성하면 됨

`split` 함수에 대한 테스트와 맥락적 의미구조 계산기 `stepC` 함수에 대한 테스트를 각각 2개 이상씩 작성해야 함

In [5]:
type Ctx = Expr -> Expr -- 타입 별명으로 긴 타입을 한 단어로 줄여 표시
instance Show Ctx where
    show ctx = show (ctx (I minBound))

In [6]:
-- 아래 split 함수를 문제에서 요구하는 대로 변경하시오
split :: Expr -> [(Ctx,Expr)]
split (I n)       = [(\e-> e, I n)]
split (Add e1 e2) = 
                 [(\e-> Add e1 (ctx e), e) | (ctx,e) <- split e2]
                 ++ [(\e-> Add (ctx e) e2, e) | (ctx,e) <- split e1]
                 ++ [(\e-> e, Add e1 e2)]

In [7]:
-- split 함수에 대한 적절한 테스트 2개 이상 작성
e0 = (I 3)
e1 = (Add (I 1) (I 2))
e2 = (Add (I 2) (Add (I 2) (I 3)))
e3 = (Add (Add (I 3) (I 4)) (Add (I 5) (I 6)))

split e0
split e1
split e2
split e3

[(I (-9223372036854775808),I 3)]

[(Add (I 1) (I (-9223372036854775808)),I 2),(Add (I (-9223372036854775808)) (I 2),I 1),(I (-9223372036854775808),Add (I 1) (I 2))]

[(Add (I 2) (Add (I 2) (I (-9223372036854775808))),I 3),(Add (I 2) (Add (I (-9223372036854775808)) (I 3)),I 2),(Add (I 2) (I (-9223372036854775808)),Add (I 2) (I 3)),(Add (I (-9223372036854775808)) (Add (I 2) (I 3)),I 2),(I (-9223372036854775808),Add (I 2) (Add (I 2) (I 3)))]

[(Add (Add (I 3) (I 4)) (Add (I 5) (I (-9223372036854775808))),I 6),(Add (Add (I 3) (I 4)) (Add (I (-9223372036854775808)) (I 6)),I 5),(Add (Add (I 3) (I 4)) (I (-9223372036854775808)),Add (I 5) (I 6)),(Add (Add (I 3) (I (-9223372036854775808))) (Add (I 5) (I 6)),I 4),(Add (Add (I (-9223372036854775808)) (I 4)) (Add (I 5) (I 6)),I 3),(Add (I (-9223372036854775808)) (Add (I 5) (I 6)),Add (I 3) (I 4)),(I (-9223372036854775808),Add (Add (I 3) (I 4)) (Add (I 5) (I 6)))]

In [8]:
stepB :: Expr -> [Expr]
stepB (I n1 `Add` I n2) = [I n] where n = n1 + n2
stepB _                 = []

stepC :: Expr -> [Expr]
stepC e0 = [ctx e' | (ctx,e) <- split e0, e' <- stepB e]

In [9]:
-- 교재 7장의 60번 셀의 내용으로 테스트
hat f = \es -> nub(concat [f e | e <- es])

e0 = (Add (I 1 `Add` I 2) (I 3 `Add` I 4))
split e0
mapM_ print it
mapM_ print(takeWhile (not . null) (iterate(hat stepC) [e0]))

[(Add (Add (I 1) (I 2)) (Add (I 3) (I (-9223372036854775808))),I 4),(Add (Add (I 1) (I 2)) (Add (I (-9223372036854775808)) (I 4)),I 3),(Add (Add (I 1) (I 2)) (I (-9223372036854775808)),Add (I 3) (I 4)),(Add (Add (I 1) (I (-9223372036854775808))) (Add (I 3) (I 4)),I 2),(Add (Add (I (-9223372036854775808)) (I 2)) (Add (I 3) (I 4)),I 1),(Add (I (-9223372036854775808)) (Add (I 3) (I 4)),Add (I 1) (I 2)),(I (-9223372036854775808),Add (Add (I 1) (I 2)) (Add (I 3) (I 4)))]

(Add (Add (I 1) (I 2)) (Add (I 3) (I (-9223372036854775808))),I 4)
(Add (Add (I 1) (I 2)) (Add (I (-9223372036854775808)) (I 4)),I 3)
(Add (Add (I 1) (I 2)) (I (-9223372036854775808)),Add (I 3) (I 4))
(Add (Add (I 1) (I (-9223372036854775808))) (Add (I 3) (I 4)),I 2)
(Add (Add (I (-9223372036854775808)) (I 2)) (Add (I 3) (I 4)),I 1)
(Add (I (-9223372036854775808)) (Add (I 3) (I 4)),Add (I 1) (I 2))
(I (-9223372036854775808),Add (Add (I 1) (I 2)) (Add (I 3) (I 4)))

[Add (Add (I 1) (I 2)) (Add (I 3) (I 4))]
[Add (Add (I 1) (I 2)) (I 7),Add (I 3) (Add (I 3) (I 4))]
[Add (I 3) (I 7)]
[I 10]

In [10]:
-- 교재 7장의 60번 셀과 마찬가지 방식으로 다른 산술식(Expr)으로 하나 더 테스트
e0 = (Add (I 2) (I 3 `Add` I 4))
mapM_ print(takeWhile (not . null) (iterate(hat stepC) [e0]))

[Add (I 2) (Add (I 3) (I 4))]
[Add (I 2) (I 7)]
[I 9]

## 문제3. 닫힌(closed)람다식을 드브루인 인덱스 형식으로 변환

In [11]:
-- 관련 코드는 9장에 있음
type Nm = String -- 변수 이름은 문자열로
data LExpr = Var Nm           -- x
           | Abs Nm LExpr     -- (λx.e)
           | App LExpr LExpr  -- (e1 e2)
           deriving (Eq, Ord)
           
instance Show LExpr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Abs x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay LExpr where display = dTeX

In [12]:
Abs "x" (Abs "y" (App (Var "x") (Abs "z"  (Var "y"))))

In [13]:
Abs "x" . Abs "y" $ Var "x" `App` Abs "z" (Var "y") -- 연산자 (.), ($) 및 역따옴표 중위 표현 활용

In [14]:
["x","y","z","w"] \\ ["y","z"] -- 차집합 연산

["x","w"]

### 문제3-a. (람다식의 자유변수들을 계산) 20점
람다식에 포함된 모든 자유변수들을 원소로 하는 리스트를 계산하는 `fv` 함수를 작성하면, 이를 활용하여 람다식이 닫힌식인지 판별하는 `closed` 함수를 간단히 작성할 수 있다.

`fv` 함수를 작성하고, `fv`를 테스트하고, 또 `fv`를 활용해 이미 정의된 `closed` 함수도 테스트하라.

In [15]:
fv :: LExpr -> [Nm] -- 자유변수의 리스트를 계산하는 함수
fv (Var x) = [x]
fv (Abs x e) = [e' | e' <- fv e] \\ [x]
fv (App e1 e2) = [e1' | e1' <- fv e1] ++ [e2' | e2' <- fv e2]

In [16]:
-- 두 개 이상의 람다식(LExpr)에 대해 fv 테스트
e0 = App (Abs "x" (App (Var "x") (Var "y"))) (Var "z")
e1 = App (Abs "x" (Abs "y" (App (Var "x") (Var "y")))) (Var "z")
fv e0
fv e1

["y","z"]

["z"]

In [17]:
closed :: LExpr -> Bool
closed e  =  fv e == []  -- 자유변수가 없는 식이 닫힌식이다

In [18]:
-- fv로 정의된 closed가 제대로 동작하는지 테스트 두 개 이상 (True인 경우 최소 하나 이상, False인 경우 최소 하나 이상)
e0 = Abs "x" (Abs "y" (Var "x"))
e1 = Abs "x" (Var "y")

closed e0
closed e1

True

False

## 문제 3-b. (닫힌 람다식을 드부루인 인덱스 형식으로 변환하는 함수 작성) 35점

In [19]:
-- 이 코드는 8장에서 가져옴
data DExpr = DI Int | DAbs DExpr | DApp DExpr DExpr deriving (Eq,Ord)

-- GHC가 자동으로 deriving하는 Show 인스턴스가 아닌  더 간략한 출력을 위해
instance Show DExpr where   -- 직접 작성한 DExpr에 대한 Show 인스턴스
  showsPrec _ (DI i) = showString (show i)
  showsPrec p (DAbs e) = showParen (p > 1) $
              showString "\\" . showsPrec 1 e
  showsPrec p (DApp e1 e2) = showParen (p > 9) $
              showsPrec 9 e1 . showString " " . showsPrec 10 e2

In [20]:
DAbs (DAbs (DApp (DI 2) (DAbs (DI 2)))) -- 𝜆𝑥.𝜆𝑦.𝑥 (𝜆𝑧.𝑦)

\\2 (\2)

In [21]:
DAbs . DAbs $ DI 2 `DApp` DAbs (DI 2) -- 연산자 (.), ($) 및 역따옴표 중위 표현 활용

\\2 (\2)

In [22]:
[]
"x" : []
"y" : ["x"]
"x" : ["y,","x"]
"z" : ["x","y,","x"]

[]

["x"]

["y","x"]

["x","y,","x"]

["z","x","y,","x"]

In [23]:
[]
"x" : it
"y" : it
"x" : it
"z" : it

[]

["x"]

["y","x"]

["x","y","x"]

["z","x","y","x"]

In [24]:
elemIndex "x" ["z","x","y","x"] -- Just 1
elemIndex "y" ["z","x","y","x"] -- Just 2
elemIndex "z" ["z","x","y","x"] -- Just 0
elemIndex "w" ["z","x","y","x"] -- Nothing

Just 1

Just 2

Just 0

Nothing

In [25]:
Just i1 = elemIndex "x" ["z","x","y","x"] -- Just 1
Just i2 = elemIndex "y" ["z","x","y","x"] -- Just 2
Just i0 = elemIndex "z" ["z","x","y","x"] -- Just 0
Just ii = elemIndex "w" ["z","x","y","x"] -- Nothing

In [26]:
i1
i2
i0

1

2

0

In [27]:
ii -- Nothing은 Just ii와 매칭할 수 없으므로 에러

: 

In [28]:
lexpr2dexpr :: LExpr -> DExpr
lexpr2dexpr e = l2d [] e   -- lexpr2dexpr 함수는 l2d 함수를 활용해 정의

l2d :: [Nm] -> LExpr -> DExpr  -- 그러니까 결국 이 함수를 작성하면 됨
l2d vs (Var x) = case elemIndex x vs of
                  Nothing -> error "Error"
                  Just i -> DI (i + 1)
l2d vs (Abs x e) = DAbs(l2d (vs ++ [x]) e)
l2d vs (App e1 e2) = DApp(l2d vs e1)(l2d vs e2)

In [29]:
-- lexpr2dexpr에 대한 테스트를 3개 이상 작성하라
--   * 8장의 Y조합자를 람다식(LExpr)으로 나타내고 lexpr2dexpr 함수로 제대로 변환되는지 테스트도 반드시 하나 포함
--   * 같은 파라메터 변수 이름의 람다요약식이 여러 겹 겹쳐 있는 테스트도 반드시 하나 포함
--   * 그 외 자유롭게 자신이 작성한 테스트 하나 이상 포함
e0 = (Abs "f" (App (Abs "x" (App (Var "f") (App (Var "x") (Var "x")))) (Abs "x" (App (Var "f") (App (Var "x") (Var "x"))))))
e1 = Abs "x" (Abs "x" (Abs "x" (Var "x")))
e2 = Abs "x" (Abs "y" (App (Var "y") (Var "y")))

lexpr2dexpr e0
lexpr2dexpr e1
lexpr2dexpr e2

\(\1 (2 2)) (\1 (2 2))

\\\1

\\2 2